<a id=contents></a>

# Model building
## What are we trying to predict?


[1. Loading up the data](#target)

[2. Fitting and transforming with BERTTopic](#samp)

[3. Exploring results](#ttsplit)

[4. How do the topics compare with other data we have?](#selection)

[4.1 Model 1](#one)

[4.2 Model 2](#two)

[4.2 Model 3](#three)

[4.2 Model 4](#four)

[4.2 Model 5](#five)

[7. Conclusions and model comparison table](#conc)

In [1]:
%load_ext autoreload
%autoreload 2

from functions.topic_modelling_BertTopic import *

# from sklearn.cluster import KMeans
# import scipy.cluster.hierarchy as shc

# import matplotlib.pyplot as plt
# import seaborn as sns

# %matplotlib inline
# sns.set_style("darkgrid")

# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# import nltk
# from sklearn.linear_model import LogisticRegression
# from nltk.probability import FreqDist
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
# from sklearn.naive_bayes import MultinomialNB
# from sklearn import metrics
# from sklearn.model_selection import train_test_split, GridSearchCV
# from matplotlib import cm
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
# from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
# from sklearn.metrics import f1_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import silhouette_score
# from sklearn.cluster import KMeans
# from sklearn.decomposition import TruncatedSVD
# import string

# from nltk.corpus import stopwords
# from nltk.tokenize import RegexpTokenizer
# import re
# tokenizer = RegexpTokenizer(r'\b\w{3,}\b')
# stop_words = list(set(stopwords.words("english")))
# stop_words += list(string.punctuation)

# import warnings
# warnings.filterwarnings('ignore')



ModuleNotFoundError: No module named 'bertopic'

<a id=target ><a/> 

## 1. ETL

[LINK to table of contents](#contents)

<a id=samp ><a/> 

## 2.  Class imbalance / sampling solution / distribution
    
[LINK to table of contents](#contents)

<a id=ttsplit ><a/> 

## 3. Train-test split and model transformation
   
[LINK to table of contents](#contents)

<a id=selection></a>

## 4. Choice of model architectures

[LINK to table of contents](#contents)

<a id=one ><a/> 

## 4.1 Model 1
    
[LINK to table of contents](#contents)

<a id=two ><a/> 

## 4.2 Model 2
    
[LINK to table of contents](#contents)

<a id=three ><a/> 

## 4.3 Model 3
    
[LINK to table of contents](#contents)

<a id=four ><a/> 

## 4.4 Model 4
    
[LINK to table of contents](#contents)

<a id=five ><a/> 

## 4.5 Model 5
    
[LINK to table of contents](#contents)

<a id=conc ><a/> 

## 7. Conclusions and model comparison table
    
[LINK to table of contents](#contents)